# EDS map processing

In [ ]:
# !pip install "https://github.com/gleesonm1/chemMap/archive/refs/tags/v0.0.3.zip"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chemMap as cm
import mineralML as mm

# this allows figures to be saved as svg files with the text preserved
plt.rcParams['svg.fonttype'] = 'none'
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'


In [2]:
# loads in wt% data for each element
Samples = dict.fromkeys(['03b', '04b', '09g1', '09g2', '09g3'])
for s in Samples:
    Samples[s] = cm.loadwt('Data/LargeScaleMaps/Gabbro Samples/' + s + '/Quant/')

In [3]:
# converts to wt% for each oxide
Quant = dict.fromkeys(list(Samples.keys()))
for s in Samples:
    Quant[s] = cm.calcOxides(Samples[s], list(Samples[s].keys()))

In [4]:
# unnormalized data so calculates a total for each pixel
for s in Quant:
    Total = np.zeros(np.shape(Samples[s]['Na']))
    for e in list(Quant[s].keys()):
        Total = Total + Quant[s][e]
        
    Quant[s]['Total'] = Total

In [5]:
# sets up flat dataframe for each sample and renames FeO to FeOt
DF = {}
for s in Quant:
    Em = np.zeros((len(Quant[s]['SiO2'].flatten()),len(Quant[s].keys())))
    for i in range(len(list(Quant[s].keys()))):
        Em[:,i] = Quant[s][list(Quant[s].keys())[i]].flatten()

    df = pd.DataFrame(data = Em, columns = list(Quant[s].keys()))
    df = df.rename(columns={"FeO": "FeOt"})
    
    DF[s] = df

In [6]:
# mineralML - I did this with v0.0.0.5, might need updating?
DF_pred = {}
for s in Quant:
    df_pred_nn, probability_matrix = mm.predict_class_prob_nn(DF[s])
    DF_pred[s] = df_pred_nn

In [7]:
# set low prob to nan
for s in Quant:
    DF_pred[s].loc[DF_pred[s]['Predict_Probability'] < 0.6,'Predict_Mineral'] = "nan"

In [8]:
Minerals = {}
for s in Quant:
    Minerals[s] = list(DF_pred[s]['Predict_Mineral'].unique())
    Minerals[s].remove("nan")

In [9]:
# assign minerals to np.array with correct shape
for s in Quant:
    Quant[s]['Mineral'] = DF_pred[s]['Predict_Mineral'].values.reshape(np.shape(Quant[s]['SiO2']))
    Quant[s]['PhaseProb'] = DF_pred[s]['Predict_Probability'].values.reshape(np.shape(Quant[s]['SiO2']))

In [10]:
import pickle
with open('GalapGabbrosQuantData.pkl', 'wb') as f:
    pickle.dump(Quant, f)